In [228]:
import pandas as pd
import json
import numpy as np

f = open('happenings.json')
happenings = json.load(f)
f.close()
#print(type(happenings))
#print(happenings[0])
happenings_converted = []
for i in happenings:
    happenings_converted.append(i[0:2])

#print(happenings_converted)
#for i in happenings:
#    print(i)
df_happenings = pd.DataFrame(happenings_converted, columns = ["Date", "Title"])
#print(df_happenings.head())

df_happenings['Date'] = pd.to_datetime(df_happenings['Date'])

#print(type(df_happenings['Date'][0]))
#print(df_happenings['Date'][0].isocalendar())

week = []
year = []
for i in df_happenings['Date']:
    #print(i) 
    year.append(i.isocalendar()[0])
    week.append(i.isocalendar()[1])
    
#print(year)
#print(week)
df_happenings["Year"] = year
df_happenings["Week"] = week

#print(df_happenings.head())

In [229]:
data = pd.read_csv("combined.csv", na_values = '-')
data.fillna(0, inplace = True)

#print(data)

#combined_total = data
combined_total = data.groupby(['Year', 'Week']).sum().reset_index()
combined_total.loc[:,'Age']="all"
combined_total = combined_total.reindex(columns = ["Year", "Week", "Age","Value","Value_Covid","Value_Covid_total"])
#print(combined_total)

data = data.append(combined_total, ignore_index=True).sort_values(by=['Year', 'Week']).reset_index(drop=True)
print(data)





      Year  Week    Age    Value  Value_Covid  Value_Covid_total
0     2015     1   0-34    200.0          0.0                0.0
1     2015     1  35-59   1637.0          0.0                0.0
2     2015     1  60-79   6665.0          0.0                0.0
3     2015     1    80+  10211.0          0.0                0.0
4     2015     1    all  18713.0          0.0                0.0
...    ...   ...    ...      ...          ...                ...
2345  2023    52   0-34    190.0          1.0              587.0
2346  2023    52  35-59   1394.0         15.0             8630.0
2347  2023    52  60-79   6743.0         80.0            53861.0
2348  2023    52    80+  13495.0        157.0           116992.0
2349  2023    52    all  21822.0        253.0           180070.0

[2350 rows x 6 columns]


In [230]:
year = 2021
kalenderwochen = [1,52]
lookback = 5
show_events = True

data_filtered = data.query("Year == @year and Week >= @kalenderwochen[0] and Week <= @kalenderwochen[1]")

data_filtered_all = data_filtered[data_filtered["Age"] == "all"]
#print(data_filtered_all)
data_filtered_young = data_filtered[data_filtered["Age"] == "0-34"]
#print(data_filtered_young)
data_filtered_semi = data_filtered[data_filtered["Age"] == "35-59"]
#print(data_filtered_semi)
data_filtered_old = data_filtered[data_filtered["Age"] == "60-79"]
#print(data_filtered_old)
data_filtered_vold = data_filtered[data_filtered["Age"] == "80+"]
#print(data_filtered_vold)


happenings_filtered = df_happenings.query("Year == @year and Week >= @kalenderwochen[0] and Week <= @kalenderwochen[1]")
#print(data_filtered)
#print(happenings_filtered)


In [231]:
lookback_data = data.query("Year < @year and Year >= @year-@lookback and Week >= @kalenderwochen[0] and Week <= @kalenderwochen[1]")
print(lookback_data)

lookback_data_grouped = lookback_data.groupby(['Year','Age']).sum().reset_index().drop(columns = ['Week','Value_Covid_total'])
#print(lookback_data_grouped)

expected_value_per_age_group = lookback_data_grouped.groupby(['Age']).mean().reset_index().drop("Year", axis = 1)
#print(expected_value_per_age_group)

std_expected_value_per_age_group = lookback_data_grouped.groupby(['Age']).std().reset_index().drop("Year", axis = 1)
print(std_expected_value_per_age_group)

expected_value_per_age_group["Std"]=std_expected_value_per_age_group["Value"]
expected_value_per_age_group["Value_High"]=expected_value_per_age_group["Value"]+std_expected_value_per_age_group["Value"]
expected_value_per_age_group["Value_Low"]=expected_value_per_age_group["Value"]-std_expected_value_per_age_group["Value"]
expected_value_per_age_group.set_index("Age", inplace = True)
print(expected_value_per_age_group)


      Year  Week    Age    Value  Value_Covid  Value_Covid_total
265   2016     1   0-34    199.0          0.0                0.0
266   2016     1  35-59   1675.0          0.0                0.0
267   2016     1  60-79   6619.0          0.0                0.0
268   2016     1    80+   9974.0          0.0                0.0
269   2016     1    all  18467.0          0.0                0.0
...    ...   ...    ...      ...          ...                ...
1560  2020    52   0-34    198.0          5.0               54.0
1561  2020    52  35-59   1553.0        104.0             1060.0
1562  2020    52  60-79   7561.0        920.0             8268.0
1563  2020    52    80+  16242.0       2674.0            20207.0
1564  2020    52    all  25554.0       3703.0            29589.0

[1300 rows x 6 columns]
     Age         Value   Value_Covid
0   0-34    396.777646     24.149534
1  35-59   2014.138600    474.046411
2  60-79   4558.199568   3697.562008
3    80+  29849.780790   9036.845124
4    all  

In [232]:
import plotly.graph_objects as go

fig_all = go.Figure()
fig_all.add_trace(go.Scatter(x = data_filtered_all["Week"], y = data_filtered_all["Value"], name = "all Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_all.add_trace(go.Scatter(x = data_filtered_all["Week"], y = np.cumsum(data_filtered_all["Value"]), name = "all Cases Total", line = dict(color = 'blue', width = 1)))





#fig_all.add_hline(y=expected_value_per_age_group.loc["all", "Value"], line_color = 'red', line_width = 1, name = "Expected all Cases Total")
fig_all.add_trace(go.Scatter(x = data_filtered_all["Week"],y = np.repeat(expected_value_per_age_group.loc["all","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))


#fig_all.add_hrect(y0=expected_value_per_age_group.loc["all","Value_Low"],y1=expected_value_per_age_group.loc["all","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)


fig_all.add_trace(go.Scatter(
    x = data_filtered_all["Week"].tolist() + data_filtered_all["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["all","Value_High"]]*52 + [expected_value_per_age_group.loc["all","Value_Low"]]*52,
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))



#print(type(np.repeat(expected_value_per_age_group.loc["all","Value_High"],52) + np.repeat(expected_value_per_age_group.loc["all","Value_Low"],52)[::-1]))




fig_all.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")


#print(happenings_filtered)

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_all.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_all.show()

In [233]:


#print(data_filtered_young["Value"])
#print(np.cumsum(data_filtered_young["Value"]))

fig_young = go.Figure()
fig_young.add_trace(go.Scatter(x = data_filtered_young["Week"], y = data_filtered_young["Value"], name = "Young Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_young.add_trace(go.Scatter(x = data_filtered_young["Week"], y = np.cumsum(data_filtered_young["Value"]), name = "Young Cases Total", line = dict(color = 'blue', width = 1)))



#fig_young.add_hline(y=expected_value_per_age_group.loc["0-34", "Value"], line_color = 'red', line_width = 1, name = "Expected Young Cases Total")
fig_young.add_trace(go.Scatter(x = data_filtered_all["Week"],y = np.repeat(expected_value_per_age_group.loc["0-34","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_young.add_hrect(y0=expected_value_per_age_group.loc["0-34","Value_Low"],y1=expected_value_per_age_group.loc["0-34","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)
fig_young.add_trace(go.Scatter(
    x = data_filtered_all["Week"].tolist() + data_filtered_all["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["0-34","Value_High"]]*52 + [expected_value_per_age_group.loc["0-34","Value_Low"]]*52,
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))


fig_young.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_young.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_young.show()

#fig_total = go.Figure()
#fig_total.add_trace(go.Scatter())

In [234]:
fig_semi = go.Figure()
fig_semi.add_trace(go.Scatter(x = data_filtered_semi["Week"], y = data_filtered_semi["Value"], name = "semi Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_semi.add_trace(go.Scatter(x = data_filtered_semi["Week"], y = np.cumsum(data_filtered_semi["Value"]), name = "semi Cases Total", line = dict(color = 'blue', width = 1)))



#fig_semi.add_hline(y=expected_value_per_age_group.loc["35-59", "Value"], line_color = 'red', line_width = 1, name = "Expected semi Cases Total")
fig_semi.add_trace(go.Scatter(x = data_filtered_semi["Week"],y = np.repeat(expected_value_per_age_group.loc["35-59","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_semi.add_hrect(y0=expected_value_per_age_group.loc["35-59","Value_Low"],y1=expected_value_per_age_group.loc["35-59","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)

fig_semi.add_trace(go.Scatter(
    x = data_filtered_semi["Week"].tolist() + data_filtered_semi["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["35-59","Value_High"]]*52 + [expected_value_per_age_group.loc["35-59","Value_Low"]]*52,
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))

fig_semi.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_semi.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_semi.show()

In [235]:
fig_old = go.Figure()
fig_old.add_trace(go.Scatter(x = data_filtered_old["Week"], y = data_filtered_old["Value"], name = "old Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_old.add_trace(go.Scatter(x = data_filtered_old["Week"], y = np.cumsum(data_filtered_old["Value"]), name = "semi Cases Total", line = dict(color = 'blue', width = 1)))



#fig_old.add_hline(y=expected_value_per_age_group.loc["60-79", "Value"], line_color = 'red', line_width = 1, name = "Expected old Cases Total")
fig_old.add_trace(go.Scatter(x = data_filtered_semi["Week"],y = np.repeat(expected_value_per_age_group.loc["60-79","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_old.add_hrect(y0=expected_value_per_age_group.loc["60-79","Value_Low"],y1=expected_value_per_age_group.loc["60-79","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)
fig_old.add_trace(go.Scatter(
    x = data_filtered_old["Week"].tolist() + data_filtered_old["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["60-79","Value_High"]]*52 + [expected_value_per_age_group.loc["60-79","Value_Low"]]*52,
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))

fig_old.update_layout(title = f"Deaths  in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_old.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_old.show()

In [236]:
fig_vold = go.Figure()
fig_vold.add_trace(go.Scatter(x = data_filtered_vold["Week"], y = data_filtered_vold["Value"], name = "vold Cases per Week", line = dict(color = 'blue', width = 1,dash = 'dot')))
fig_vold.add_trace(go.Scatter(x = data_filtered_vold["Week"], y = np.cumsum(data_filtered_vold["Value"]), name = "vold Cases Total", line = dict(color = 'blue', width = 1)))


#fig_vold.add_hline(y=expected_value_per_age_group.loc["80+", "Value"], line_color = 'red', line_width = 1, name = "Expected vold Cases Total")
fig_vold.add_trace(go.Scatter(x = data_filtered_semi["Week"],y = np.repeat(expected_value_per_age_group.loc["80+","Value"],kalenderwochen[1]-kalenderwochen[0]+1), name = "expected Value", line = dict(color = 'red')))

#fig_vold.add_hrect(y0=expected_value_per_age_group.loc["80+","Value_Low"],y1=expected_value_per_age_group.loc["80+","Value_High"],fillcolor = "red", line_width = 0, opacity = 0.2)
fig_vold.add_trace(go.Scatter(
    x = data_filtered_vold["Week"].tolist() + data_filtered_vold["Week"].tolist()[::-1] ,
    y = [expected_value_per_age_group.loc["80+","Value_High"]]*52 + [expected_value_per_age_group.loc["80+","Value_Low"]]*52,
    fill = 'toself',
    fillcolor = "red",
    line_color = "red",
    name = "Varianzbereich",
    opacity=0.2
))


fig_vold.update_layout(title = f"Deaths in {year} between week {kalenderwochen[0]} and week {kalenderwochen[1]}",
                        xaxis_title = "Week",
                        yaxis_title = "Deaths")

if show_events:
    for i in happenings_filtered.index:
        #print(happenings_filtered.loc[i,"Week"])
        #print(happenings_filtered.loc[i, "Title"])
        fig_vold.add_vline(x = happenings_filtered.loc[i,"Week"], line_width = 2, line_color = "green", line_dash = "dash")

fig_vold.show()